# (실습) 판다스 활용: 통계 기초

In [1]:
import numpy as np
import pandas as pd

np.set_printoptions(precision=6, suppress=True)
PREVIOUS_MAX_ROWS = pd.options.display.max_rows # 원래 60이 기본.
pd.options.display.max_rows = 20

보스턴 데이터는 보스턴 시의 506개 구역에 대한 통계 자료를 담고 있다.
먼저 보스턴 데이터를 불러온다.

**안내사항**

보스턴 데이터를 원래는 판다스 자체에서 제공했었지만 인종차별 내용을 담고 있다는 이유로 삭제되었다.
여기서는 순수히 데이터분석 용으로만 사용하며 어떤 다른 의도도 없음을 밝힌다.
이 연습문제 또한 언젠가는 다른 데이터셋으로 대체될 것이다.

In [56]:
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

`X`는 2차원 어레이다.

In [57]:
type(X)

numpy.ndarray

In [58]:
X.ndim

2

In [59]:
X

array([[  0.00632,  18.     ,   2.31   , ...,  15.3    , 396.9    ,
          4.98   ],
       [  0.02731,   0.     ,   7.07   , ...,  17.8    , 396.9    ,
          9.14   ],
       [  0.02729,   0.     ,   7.07   , ...,  17.8    , 392.83   ,
          4.03   ],
       ...,
       [  0.06076,   0.     ,  11.93   , ...,  21.     , 396.9    ,
          5.64   ],
       [  0.10959,   0.     ,  11.93   , ...,  21.     , 393.45   ,
          6.48   ],
       [  0.04741,   0.     ,  11.93   , ...,  21.     , 396.9    ,
          7.88   ]])

`X`는 미국 보스턴 시의 구역별 통계자료를 갖고 있으며, (506, 13) 모양의 2차원 어레이다.
각각의 열(column)은 아래 특성을 가리킨다.

| 특성 | 의미 |
|:------|:---------|
| CRIM  | 구역별 1인당 범죄율 |
| ZN    | 25,000 평방 피트 이상의 주거 구역 비율 |
| INDUS | 구역별 비 소매 사업 에이커(acre) 비율 |
| CHAS  | Charles River 더미 변수(구역이 강 경계에 닿으면 1, 아니면 0) |
| NOX   | 산화 질소 농도(1000만분 율) |
| RM    | 주택 당 평균 방 수 |
| AGE   | 소유주가 살고 있는 1940년 이전에 지어진 건물 비율 |
| DIS   | 보스턴 고용 센터 다섯 곳 까지의 가중 거리 |
| RAD   | 방사형 고속도로 접근성 지수 |
| TAX   | 1만달러당 전체 가지 재산 세율 |
| PTRATIO | 구역별 학생-교사 비율 |
| B     | 1000(Bk - 0.63)^2 (Bk 구역별 흑인 비율) |
| LSTAT | 구역별 낮은 지위 인구 비율 |

**문제 1**

아래 코드가 의미하는 바를 설명하라.

In [60]:
X.shape

(506, 13)

답) 데이터의 전체 크기가 어느정도인지 보여준다 506 행에 13 열이 있다


이제 `X`를 `boston` 변수에 할당한다.

In [61]:
boston = X

열(column)별 이름, 즉 특성별 이름을 지정하면서 판다스의 데이터프레임으로 형변환한다.
특성별 이름은 다음과 같다.

In [62]:
feature_names = np.array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
                           'TAX', 'PTRATIO', 'B', 'LSTAT'], dtype='<U7')

In [63]:
boston = pd.DataFrame(X, columns=feature_names)
boston

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48


원래 결측치가 없지만 아래처럼 고의로 5개의 결측치를 입력한다.
대상은 주택당 평균 방수 특성(RM)의 100번에서 104번행이다.

In [64]:
boston['RM'][100:105] = np.nan

<ipython-input-64-62bbe732eae9>:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  boston['RM'][100:105] = np.nan


**문제 2**

보스턴 전체 데이터셋에서 수치상으로 5개의 값만 결측치임을 보여주는 표현식을 작성하라.

In [65]:
# None을 각각 적절한 코드와 표현식으로 대체하라.
missing_counts = boston.isna().sum()
assert missing_counts.sum() == 5

**문제 3**

결측치가 들어있는 행으로만 이루어진 데이터프레임을 가리키는 `boston_with_nan`을 정의하라.
부울 마스크를 이용하며, `copy()` 메서드를 적용하여 새로운 데이터프레임으로 생성한다.

In [66]:
# pass와 None을 각각 적절한 코드와 표현식으로 대체하라.

boston_with_nan = boston[boston.isna().any(axis = 1)].copy()
boston_with_nan

#assert len(boston_with_nan) == 5
assert boston_with_nan.shape[0] == 5

**문제 4**

결측치가 들어있는 행을 제외한 데이터프레임을 가리키는 `boston_without_nan`을 정의하라.
부울 마스크를 이용하며, `copy()` 메서드를 적용하여 새로운 데이터프레임으로 생성한다.

In [67]:
# pass와 None을 각각 적절한 코드와 표현식으로 대체하라.

boston_without_nan = boston[~boston.isna().any(axis = 1)].copy()
boston_without_nan
assert boston_without_nan.shape == (501, 13)

**문제 5**

`boston_with_nan`이 가리키는 데이터프레임에 사용된 결측치(`NaN`)를 모두 해당 특성의 평균값(mean)으로 대체하라. 즉,
`NaN`이 실제로 대체되어야 한다.

In [68]:
# boston_with_nan 데이터프레임에 포함된 모든 결측치를 평균값으로 대체해야 함.
# None을 적절한 표현식으로 대체할 것.

rm_mean = boston.mean()
boston_with_nan.fillna(rm_mean, inplace=True)
boston_with_nan

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
100,0.14866,0.0,8.56,0.0,0.52,6.28305,79.9,2.7778,5.0,384.0,20.9,394.76,9.42
101,0.11432,0.0,8.56,0.0,0.52,6.28305,71.3,2.8561,5.0,384.0,20.9,395.58,7.67
102,0.22876,0.0,8.56,0.0,0.52,6.28305,85.4,2.7147,5.0,384.0,20.9,70.80,10.63
103,0.21161,0.0,8.56,0.0,0.52,6.28305,87.4,2.7147,5.0,384.0,20.9,394.47,13.44
104,0.13960,0.0,8.56,0.0,0.52,6.28305,90.0,2.4210,5.0,384.0,20.9,392.69,12.33


In [69]:
# boston_with_nan 데이터프레임에 결측치가 없음을 입증해야 함.
# None 을 적절한 표현식으로 대체할 것.

assert boston_with_nan.isna().all().all() == False

**문제 6**

`AGE` 특성의 값들에서 소수점 이하를 버리고 남은 정수값(int)만 이용하는 `AGE_Year` 특성을 추가하라. 즉, 소수점 이하를 버리고 정수형(int)으로 형변환된 값을 이용해야 한다.

In [70]:
# None을 적절한 표현식으로 대체하라.
# AGE_Year 특성 추가하기

boston["AGE_year"] = boston["AGE"].astype(np.int32)
#boston["AGE_year"] = boston["AGE"].astype("Int64") #NaN 값이 있어도 허용하는 변환이 가능한 코드
#boston["AGE_year"] = pd.to_numeric(boston["AGE"], errors="coerce") #문자데이터가 포함된 경우에도 변환이 가능한 코드
#boston.AGE_year.dtype

assert boston.AGE_year.dtype == 'int32'

**문제 7**

`AGE` 특성을 삭제하라.

In [71]:
# pass와 None을 각각 적절한 코드와 표현식으로 대체하라.

boston = boston.drop(columns=['AGE'])

assert ('AGE' in boston.columns) == False